In [29]:
###  Small sledgehammer to calculate bond lenghts,   ###
###  coordination numbers, averages etc. Currently   ###
###  only suitable for cubic systems.                ###
###  Author : Adam Symington                         ###
###  Contact : ars44@bath.ac.uk                      ###
########################################################

## Import Modules ##
import numpy as np
import math as math
from math import pi
pi = math.pi
f = open('CONTCAR','r')

## read in structure name ##
line = f.readline().split()
name = ''
for i in range(len(line)):
    name = name + line[i] + ' '

## read in scale factor  ##
scale = str(f.readline().split()[0])

## read in unit cell lattice vectors ##
###############  To Do ##################################
###############  Rewrite this ###########################
lat = np.zeros((3,3))
for i in range(3):
    line = f.readline().split()
    for j in range(3):
        lat[i,j]=float(line[j])
x = lat[0,0], lat[0,1], lat[0,2]
x = np.asarray(x)
y = lat[1,0], lat[1,1], lat[1,2]
y = np.asarray(y)
z = lat[2,0], lat[2,1], lat[2,2]
z = np.asarray(z)

##########   This Function Calculates  and returns the lenght of an edge
def get_lenghts(A):
    if A[0] == 0.0 and A[1] == 0.0:
        lenght = A[2]
    elif A[0] == 0.0 and A[2] == 0.0:
        lenght = A[1]
    elif A[1] == 0.0 and A[2] == 0.0:
        lenght = A[0]
    elif A[0] != 0.0 and A[1] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[1] ** 2))
    elif A[1] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[1] ** 2) + (A[2] ** 2))
    elif A[0] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[2] ** 2))
    elif A[0] != 0.0 and A[1] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[1] ** 2) + (A[2] ** 2))
    return(lenght)
        
A = x
x_vec = get_lenghts(A)
A = y 
y_vec = get_lenghts(A)
A = z
z_vec = get_lenghts(A)
alpha = 90
beta = 90
gamma = 60
Coord_system = "Hexagonal" 

alpha = np.deg2rad(alpha)
beta = np.deg2rad(beta)
gamma = np.deg2rad(gamma)
cosa = np.cos(alpha)
sina = np.sin(alpha)
cosb = np.cos(beta)
sinb = np.sin(beta)
cosg = np.cos(gamma)
sing = np.sin(gamma)
volume = 1.0 - cosa**2.0 - cosb**2.0 - cosg**2.0 + 2.0 * cosa * cosb * cosg
volume = np.sqrt(volume)
r = np.zeros((3, 3))
r[0, 0] = x_vec
r[0, 1] = y_vec * cosg
r[0, 2] = z_vec * cosb
r[1, 1] = y_vec * sing
r[1, 2] = z_vec * (cosa - cosb * cosg) / sing
r[2, 2] = z_vec * volume / sing
  
#### Calculate X and Y scalers ###########
y_scale =y_vec * (math.sin(gamma))
x_scale = (math.sqrt((y_vec ** 2) - (y_scale ** 2)))
print(x_scale, y_scale)
    
## read in number of atom types, number of atoms of each type ##

atom_types = f.readline().split()
atom_nums = f.readline().split()
atom_nums = list(map(int, atom_nums))
total_atoms = sum(atom_nums)

#read in atom coordinate convention
convention = f.readline().split()[0]

#read in atomic positions
Coord = np.zeros((total_atoms,3))
for i in range(total_atoms):
    line = f.readline().split()
    for j in range(3):
        Coord[i,j] = float(line[j])

if convention == "Direct":
    Coord_Cart = ([])
    for row in Coord:
        Coord = np.dot(r, row)
        Coord_Cart = np.append(Coord_Cart, Coord)

Coord_Cart = np.reshape(Coord_Cart, (total_atoms, 3))

x_coords = Coord_Cart[:,[0]]
y_coords = Coord_Cart[:,[1]]
z_coords = Coord_Cart[:,[2]]


## Declare what bonds are to be counted ##
Bond_check = input("Enter the atom you want to calculate bond lenghts for: ")
Bond_check = str(Bond_check)
Coord_spec = input("Enter the coordinating species: ")
Coord_spec = str(Coord_spec)
Max_lenght = input("Enter the Maximum bond lenght to be considered: ")
Max_lenght = float(Max_lenght)

## Read Coordinating Species ##
def get_x_X(Coord_spec):
    for i, j in enumerate(atom_types):
        if j == Coord_spec:
            lower_lim = sum(atom_nums[:i])
            n_X = (atom_nums[i])
            upper_lim = lower_lim + n_X
            Coord_spec_x_cart = x_coords[lower_lim:upper_lim]
            Coord_spec_y_cart = y_coords[lower_lim:upper_lim]
            Coord_spec_z_cart = z_coords[lower_lim:upper_lim]
    
    return Coord_spec_x_cart, Coord_spec_y_cart, Coord_spec_z_cart


##  Read in Species at center of Coordination sphere ##
def get_x_M(Bond_check):
    M_x_cart = np.array([])
    M_y_cart = np.array([])
    M_z_cart = np.array([])
    for i, j in enumerate(atom_types):
        if j == Bond_check:
            lower_lim = sum(atom_nums[:i])
            n_M = (atom_nums[i])
            upper_lim = lower_lim + n_M
            ran = upper_lim - lower_lim
            Top_M_x_cart = x_coords[lower_lim:upper_lim]
            Top_M_y_cart = y_coords[lower_lim:upper_lim]
            Top_M_z_cart = z_coords[lower_lim:upper_lim]
            M_x_cart = np.append(M_x_cart, Top_M_x_cart)
            M_y_cart = np.append(M_y_cart, Top_M_y_cart)
            M_z_cart = np.append(M_z_cart, Top_M_z_cart)
    return M_x_cart, M_y_cart, M_z_cart


## Bond Lenght Calculations ##

def differences():
    ABx =  Coord_spec_x_cart[j] - M_x_cart[i] 
    ABy = Coord_spec_y_cart[j]  - M_y_cart[i]
    ABz = Coord_spec_z_cart[j] -  M_z_cart[i]
    return ABx, ABy, ABz

def three_body():
    ABx = (Coord_spec_x_cart[j] - M_x_cart[i])
    ABy = (Coord_spec_y_cart[j] - M_y_cart[i])
    ABz = (Coord_spec_z_cart[j] - M_z_cart[i])
                
    CBx = (Coord_spec_x_cart[k] - M_x_cart[i])
    CBy = (Coord_spec_y_cart[k] - M_y_cart[i])
    CBz = (Coord_spec_z_cart[k] - M_z_cart[i])
    return ABx, ABy, ABz, CBx, CBy, CBz

            
def PBC_orthogonal(M, N, O):
    
    if (M) > (x_vec*0.5):
        M = M - x_vec 
    elif (-M) > (x_vec*0.5):
        M = M + x_vec
             
    if (N) > (y_vec*0.5):
        N = N - y_vec
    elif (-N) > (y_vec*0.5):
        N = N + y_vec
            
    if (O) > (z_vec*0.5):
        O = O - z_vec
    elif (-O) > (z_vec*0.5):
        O = O + z_vec
    return M, N, O


def PBC_hexagonal(M, N, O):    
  
    if M > (x_vec * 0.5) and N > (y_vec * 0.5):
        N = N - y_scale
        M = M - x_scale
    elif -M > (x_vec * 0.5) and -N > (y_vec * 0.5):
        N = N + y_scale
        M = M + x_scale
        
    if M < (x_vec * 0.5) and N > (y_vec * 0.5):
        N = N - y_scale
        M = M - x_scale
    elif -M < (x_vec * 0.5) and -N > (y_vec * 0.5):
        N = N + y_scale
        M = M + x_scale
      
    if (M) > (x_vec * 0.5):
        M = M - x_vec
    elif -(M) > (x_vec * 0.5):
        M = M + x_vec
        
    if O > (z_vec * 0.5):
        O = O - z_vec
    elif (-O) > (z_vec * 0.5):
        O = O + z_vec
        
    return M, N, O

def get_bond_lenght (ABx, ABy, ABz):       
    Bond_lenght = math.sqrt((ABx ** 2) + (ABy ** 2) + (ABz ** 2)) 
    return(Bond_lenght)

                

M_x_cart, M_y_cart, M_z_cart = get_x_M(Bond_check)
Coord_spec_x_cart, Coord_spec_y_cart, Coord_spec_z_cart  = get_x_X(Coord_spec)
n_M = np.size(M_x_cart)
n_X = np.size(Coord_spec_x_cart)
Bond_lenght_array = np.array([])

for i in range(0, n_M):
    M_array = np.array([])
    for j in range(0, n_X):
        ABx, ABy, ABz = differences() 
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            Bond_lenght = bond_lenghts_orthogonal(ABx, ABy, ABz)
        if Bond_lenght < 3.0:
            M_array = np.append(M_array, Bond_lenght)
    if M_array.size:
        print()
        print(Bond_check, "-", i + 1, M_array)
        print()
        print("Average Bond Lenght for ", Bond_check, "-", i + 1, np.average(M_array))
        print("Range in Bond Lenght:", np.amax(M_array), " - ", np.amin(M_array))
        print()
        print("Coordination Number - ", M_array.size)
        print()
    elif not M_array.size:
        print(Bond_check, "-", i + 1)
        print("There are no", Coord_spec, "within", Max_lenght, "of", Bond_check, "-", i+1)
        print("Try increasing the Bond lenght parameter")
    Bond_lenght_array = np.append(Bond_lenght_array, M_array)
print()
print("Average", Bond_check, "-", Coord_spec, "Bond Lenght: ", np.average(Bond_lenght_array))
print("Range in Bond Length Values: ", np.amax(Bond_lenght_array), " - ", np.amin(Bond_lenght_array))
print()
print("Standard Deviation: ", np.std(Bond_lenght_array))
print()
print("   #######    Hydrogen Bond Section    ########")
print()
print("Definition of Hydrogen Bonding Strength")
print("Strong H-Bond = 1.2- 1.6")
print("Medium H-Bond = 1.6 - 2.2")
print("Weak H-Bond = 2.2 - 3.0")
print()


Coord_spec = "O"
Bond_check = "H"
M_x_cart, M_y_cart, M_z_cart = get_x_M(Bond_check)
Coord_spec_x_cart, Coord_spec_y_cart, Coord_spec_z_cart  = get_x_X(Coord_spec)
n_M = np.size(M_x_cart)
n_X = np.size(Coord_spec_x_cart)
nOH_bonds = 0 
nH_Bonds = 0
nStrong = 0 
nMed = 0
nWeak = 0
for i in range(0, n_M):
    Strong_H_Bond_array = np.array([])
    Medium_H_Bond_array = np.array([])
    Weak_H_Bond_array = np.array([])
   # print(M_x_cart)
    for j in range(0, n_X):
        ABx, ABy, ABz = differences() 
        if Coord_system == "Hexagonal":
            Bond_lenght = bond_lenghts_hexagonal(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            Bond_lenght = bond_lenghts_orthogonal(ABx, ABy, ABz)
        if Bond_lenght < 1.2:
            nOH_bonds = nOH_bonds + 1
            for k in range(0, n_X):
                ABx, ABy, ABz, CBx, CBy, CBz = three_body()
                
                if Coord_system != "Hexagonal":   
                    ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
                    CBx, CBy, CBz = PBC_orthogonal(CBx, CBy, CBz)   

                elif Coord_system == "Hexagonal":
                    ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
                    CBx, CBy, CBz = PBC_hexagonal(CBx, CBy, CBz) 
                
                X_diff = (ABx * CBx)
                Y_diff = (ABy * CBy)
                Z_diff = (ABz * CBz)
                    
                Bond_lenght_2 = (math.sqrt((CBx ** 2) + (CBy ** 2) + (CBz ** 2)))
                Bond_lenght_2 = float(Bond_lenght_2)
                OHOangle = (X_diff + Y_diff + Z_diff)
                OHOangle = float(OHOangle)
                OHOangle = (OHOangle/(Bond_lenght * Bond_lenght_2))
                OHOangle = np.around(OHOangle, decimals=12)
                OHOangle = float(OHOangle)
                OHOangle = math.acos(OHOangle)
                OHOangle = OHOangle*(180/pi)
                Bond_lenght_2 = (np.around(Bond_lenght_2, decimals=3))
                
                if OHOangle > 90 and Bond_lenght_2 < 1.6 and Bond_lenght_2 > 1.2:
                    print("H -", i+1, "-", "Strong H-Bond -", Bond_lenght_2)
                    nH_Bonds = nH_Bonds + 1
                    nStrong = nStrong + 1
                    Strong_H_Bond_array = np.append(Strong_H_Bond_array, Bond_lenght_2)
                elif OHOangle > 90 and Bond_lenght_2 < 2.2 and Bond_lenght_2 > 1.6:
                    print("H -", i+1, "-", "Medium H-Bond -", Bond_lenght_2)
                    nH_Bonds = nH_Bonds + 1
                    nMed = nMed + 1
                    Medium_H_Bond_array = np.append(Medium_H_Bond_array, Bond_lenght_2)
                elif OHOangle > 90 and Bond_lenght_2 < 3.0 and Bond_lenght_2 > 2.2:
                    print("H -", i+1 , "-", "Weak H-bond -", Bond_lenght_2)
                    nH_Bonds = nH_Bonds + 1
                    nWeak = nWeak + 1
                    Weak_H_Bond_array = np.append(Weak_H_Bond_array, Bond_lenght_2)
#print(Weak_H_bond_array)
print()
print("Total number of Hydrogen Bonds", nH_Bonds) 
print("Total number of Strong Hydrogen Bonds = ", nStrong)
print("Total number of Medium Hydrogen Bonds = ", nMed)
print("Total number of Weak Hydrogen Bonds = ", nWeak)

f.close()

5.821698517939421 10.083477619419506
Enter the atom you want to calculate bond lenghts for: Gd
Enter the coordinating species: O
Enter the Maximum bond lenght to be considered: 3

Gd - 1 [ 2.36075887  2.55045684  2.5115012   2.31187751  2.31898143  2.22217552
  2.37861878]

Average Bond Lenght for  Gd - 1 2.37919573519
Range in Bond Lenght: 2.5504568419  -  2.22217551741

Coordination Number -  7


Gd - 2 [ 2.34954236  2.38669627  2.22570637  2.38147136  2.39410234  2.27347474]

Average Bond Lenght for  Gd - 2 2.33516557212
Range in Bond Lenght: 2.39410233576  -  2.22570636602

Coordination Number -  6


Gd - 3 [ 2.39806251  2.38288275  2.23445822  2.35592555  2.35558954  2.2646661 ]

Average Bond Lenght for  Gd - 3 2.33193077844
Range in Bond Lenght: 2.3980625076  -  2.23445822023

Coordination Number -  6


Gd - 4 [ 2.19431428  2.33508422  2.27651454  2.39933804  2.33013639  2.43228141]

Average Bond Lenght for  Gd - 4 2.32794481305
Range in Bond Lenght: 2.43228141269  -  2.194314280